## 재앙적 망각 모니터링용 검증 데이터셋 생성

이 노트북은 `AffectNet`과 `FER2013` 데이터셋의 **test 폴더**에 있는 이미지들을 사용하여, `07_finetuning.py` 스크립트에서 사용할 `general_val_set.csv` 파일을 생성합니다.

**전략:**
1. 각 데이터셋의 `test` 폴더 경로를 설정합니다.
2. 폴더를 탐색하여 이미지의 경로와 감정 라벨(폴더명)을 수집합니다.
3. Hard Label(예: 'happy')을 One-hot Encoding 방식의 Soft Label(예: `soft_happy`=1.0, 나머지=0.0)로 변환합니다.
4. `06_softlabel_dataset_resplit.csv`와 동일한 형태로 DataFrame을 만들어 CSV 파일로 저장합니다.

### 1. 라이브러리 임포트 및 설정

In [1]:
import os
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

### 2. 데이터셋 경로 및 설정

In [2]:
# 데이터셋의 기본 경로
base_data_path = './Data/'

# 모니터링에 사용할 test 폴더 경로
dataset_paths = {
    'AffectNet': os.path.join(base_data_path, 'AffectNet/Test'),
    'FER2013': os.path.join(base_data_path, 'FER2013/test')
}

# 백본 모델이 사용하는 클래스 (07_finetuning.py 참고)
# 순서가 중요합니다.
backbone_classes = ['anger', 'contempt', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

print("데이터셋 경로:", dataset_paths)
print("백본 클래스:", backbone_classes)

데이터셋 경로: {'AffectNet': './Data/AffectNet/Test', 'FER2013': './Data/FER2013/test'}
백본 클래스: ['anger', 'contempt', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


### 3. 이미지 파일 탐색 및 데이터 리스트 생성

In [4]:
image_data_list = []

# 각 데이터셋 경로에 대해 반복
for dataset_name, path in dataset_paths.items():
    print(f"Processing {dataset_name}...")

    # 감정 라벨 폴더 목록 가져오기
    emotion_folders = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    
    for emotion in tqdm(emotion_folders, desc=f'{dataset_name} Emotions'):
        # 감정 라벨 이름 클리닝 및 확인 (소문자로 변환)
        cleaned_emotion = emotion.lower()
        
        if cleaned_emotion not in backbone_classes:
            print(f"[Warning] Skipping emotion '{emotion}' as it is not in backbone_classes.")
            continue
            
        # 이미지 파일 경로 탐색 (jpg, png 등 다양한 확장자 지원)
        image_files = glob(os.path.join(path, emotion, '*.jpg')) + \
                      glob(os.path.join(path, emotion, '*.png')) + \
                      glob(os.path.join(path, emotion, '*.jpeg'))
        
        for img_path in image_files:
            # One-hot 인코딩 방식의 soft label 생성
            soft_label_vector = {f'soft_{cls}': 0.0 for cls in backbone_classes}
            soft_label_vector[f'soft_{cleaned_emotion}'] = 1.0
            
            # CSV 한 행에 들어갈 데이터 딕셔너리 생성
            row = {
                'phase': 'val',
                'category': cleaned_emotion,
                'filename': os.path.basename(img_path),
                'img_path': img_path.replace('\\', '/'), # 경로 구분자 통일
                'exists': True,
                'annot_A': np.nan,
                'annot_B': np.nan,
                'annot_C': np.nan,
            }
            
            # soft label 값을 row에 추가
            row.update(soft_label_vector)
            image_data_list.append(row)

print(f"총 {len(image_data_list)}개의 이미지 파일을 수집했습니다.")

Processing AffectNet...


AffectNet Emotions:   0%|          | 0/8 [00:00<?, ?it/s]

AffectNet Emotions: 100%|██████████| 8/8 [00:00<00:00, 52.63it/s]


Processing FER2013...


FER2013 Emotions:   0%|          | 0/7 [00:00<?, ?it/s]

[Warning] Skipping emotion 'angry' as it is not in backbone_classes.


FER2013 Emotions: 100%|██████████| 7/7 [00:00<00:00, 80.36it/s]

총 20738개의 이미지 파일을 수집했습니다.


### 4. DataFrame 생성 및 CSV 저장

In [6]:
# 리스트를 DataFrame으로 변환
df_general_val = pd.DataFrame(image_data_list)

# 컬럼 순서 정의 (기존 파일과 동일하게)
final_columns = [
    'phase', 'category', 'filename', 'img_path', 'exists', 
    'annot_A', 'annot_B', 'annot_C'
] + [f'soft_{cls}' for cls in backbone_classes]

df_general_val = df_general_val[final_columns]

# 결과 확인
print(f"생성된 DataFrame 형태: {df_general_val.shape}")
print("Category 분포:")
print(df_general_val['category'].value_counts())

# 파일 저장
output_filename = '07_general_val_set.csv'
df_general_val.to_csv(output_filename, index=False)

print(f"{output_filename} 파일이 성공적으로 생성되었습니다.")
df_general_val.head()

생성된 DataFrame 형태: (20738, 16)
Category 분포:
category
happy       4478
neutral     3601
sad         2831
surprise    2751
fear        2688
anger       1718
disgust     1359
contempt    1312
Name: count, dtype: int64
07_general_val_set.csv 파일이 성공적으로 생성되었습니다.


,phase,category,filename,img_path,exists,annot_A,annot_B,annot_C,soft_anger,soft_contempt,soft_disgust,soft_fear,soft_happy,soft_neutral,soft_sad,soft_surprise
0,val,disgust,image0030361.jpg,./Data/AffectNet/Test/disgust/image0030361.jpg,True,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,val,disgust,image0035046.jpg,./Data/AffectNet/Test/disgust/image0035046.jpg,True,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,val,disgust,image0020592.jpg,./Data/AffectNet/Test/disgust/image0020592.jpg,True,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,val,disgust,image0025565.jpg,./Data/AffectNet/Test/disgust/image0025565.jpg,True,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,val,disgust,image0034488.jpg,./Data/AffectNet/Test/disgust/image0034488.jpg,True,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
